In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from numpy import nan as NA


Renaming the columns and putting datasets together

In [5]:
list_of_df = []
for file in os.listdir("/Users/jennyjang/Desktop/CS Project/datasets/"):
    if file != '.DS_Store':
        if '.csv' in file:
            df = pd.read_csv(os.path.join("/Users/jennyjang/Desktop/CS Project/datasets", file))
        else:
            df = pd.read_csv(os.path.join("/Users/jennyjang/Desktop/CS Project/datasets", file), sep=",")
        df.rename(columns = {'Start date':'started_at', 
                              'End date':'ended_at',
                              'Start station':'start_station_name',
                              'Start station number':'start_station_id',
                              'End station number':'end_station_id',
                              'End station':'end_station_name',
                              'Bike number':'ride_id',
                              'Member type':'member_casual'
                              }, inplace = True)
        df['fileid']=df['started_at'].str[0:7]
        df['year']=df['started_at'].str[0:4]
        df['month']=df['started_at'].str[6:8]
        df['member_casual']=df['member_casual'].str.lower()
        list_of_df.append(df)
df = pd.concat(list_of_df) #Puts all DF together

df["id"] = df.index + 1 #Create unique stable id for each observation
print(df.columns)
print(df.shape)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Duration', 'started_at', 'ended_at', 'start_station_id',
       'start_station_name', 'end_station_id', 'end_station_name', 'ride_id',
       'member_casual', 'fileid', 'year', 'month', 'Unnamed: 0',
       'rideable_type', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'id',
       'temp_start_lat', 'temp_start_lng', 'temp_end_lat', 'temp_end_lng'],
      dtype='object')
(11475638, 23)


In [5]:
df = df[df["start_station_id"] != 'MTL-ECO5-03']
temp = df[["start_station_id","start_lat","start_lng"]]
temp["start_station_id"] = temp["start_station_id"]
df_start_lat_lng = temp[temp["start_lat"] != 'nan']
df_start_lat_lng["start_station_id"] = df_start_lat_lng["start_station_id"].astype(float).astype(str)
df_start_lat_lng["start_station_id"] = df_start_lat_lng["start_station_id"].str.split('.').str[0]
df_start_lat_lng = df_start_lat_lng.groupby("start_station_id").mean().reset_index()

df = df[df["end_station_id"] != 'MTL-ECO5-03']
temp = df[["end_station_id","end_lat","end_lng"]]
temp["end_station_id"] = temp["end_station_id"]
df_end_lat_lng = temp[temp["end_lat"] != 'nan']
df_end_lat_lng["end_station_id"] = df_end_lat_lng["end_station_id"].astype(float).astype(str)
df_end_lat_lng["end_station_id"] = df_end_lat_lng["end_station_id"].str.split('.').str[0]
df_end_lat_lng = df_end_lat_lng.groupby("end_station_id").mean().reset_index()


<ipython-input-5-de7d10793c45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["start_station_id"] = temp["start_station_id"]
<ipython-input-5-de7d10793c45>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["end_station_id"] = temp["end_station_id"]


In [6]:
dict_start_lat_lng = df_start_lat_lng.set_index('start_station_id').T.to_dict('list')
dict_end_lat_lng = df_end_lat_lng.set_index('end_station_id').T.to_dict('list')

In [8]:
dict_start_lat_lng
dict_end_lat_lng

{'0': [nan, nan],
 '31000': [38.8589697184318, -77.05322864566992],
 '31001': [38.857252148847415, -77.05335301968522],
 '31002': [38.85642876354931, -77.04923456406321],
 '31003': [38.86028445215177, -77.04957006476029],
 '31004': [38.85786759428505, -77.05949021371399],
 '31005': [38.862287816413016, -77.05993565732342],
 '31006': [38.86367176241578, -77.06330241697007],
 '31007': [38.85740642457152, -77.05113235027416],
 '31009': [38.848488043112525, -77.05147341097081],
 '31010': [38.842613615396616, -77.0502011286223],
 '31011': [38.8532803827816, -77.04978832780324],
 '31012': [38.85068240005385, -77.05150984494931],
 '31013': [38.85438361880703, -77.05341172932293],
 '31014': [38.89733246486175, -77.07098653775525],
 '31015': [38.89461302413212, -77.07230156347926],
 '31016': [38.89345139838152, -77.07637215281194],
 '31017': [38.89170236373495, -77.08459242616718],
 '31018': [38.892165906654604, -77.07938137971207],
 '31019': [38.8890096014243, -77.09248686282992],
 '31020': [3

In [6]:
def fillLatNa(row):
    if row["start_station_id"] == 'nan':
        return [row["start_lat"], row["start_lng"]]
    elif str(row["start_station_id"]).split('.')[0] in dict_start_lat_lng.keys():
        return dict_start_lat_lng[str(row["start_station_id"]).split('.')[0]]
    else:
        print(row)

lat_long_apply = df.apply(fillLatNa, axis=1)

NameError: name 'dict_start_lat_lng' is not defined

In [10]:
lat_long_apply

0          [38.91209662213295, -77.03868857241436]
1          [38.88737359686413, -77.00195770159645]
2          [38.91209662213295, -77.03868857241436]
3         [38.938741243129506, -77.08718081481568]
4          [38.90864431116117, -77.01236611713998]
                            ...                   
158125     [38.90573073109983, -77.02226395599776]
158126     [38.90859554399085, -77.03230763506622]
158127     [38.91780661088507, -77.03207932431052]
158128     [38.88408977885549, -76.99517967895706]
158129      [38.90659875790153, -77.0514937164447]
Length: 5737818, dtype: object

In [11]:
def fillLatNaEnd(row):
    if row["end_station_id"] == 'nan':
        return [row["end_lat"], row["end_lng"]]
    elif str(row["end_station_id"]).split('.')[0] in dict_end_lat_lng.keys():
        return dict_end_lat_lng[str(row["end_station_id"]).split('.')[0]]
    else:
        print(row)

lat_long_end_apply = df.apply(fillLatNa, axis=1)

In [12]:
lat_long_end_apply

0          [38.91209662213295, -77.03868857241436]
1          [38.88737359686413, -77.00195770159645]
2          [38.91209662213295, -77.03868857241436]
3         [38.938741243129506, -77.08718081481568]
4          [38.90864431116117, -77.01236611713998]
                            ...                   
158125     [38.90573073109983, -77.02226395599776]
158126     [38.90859554399085, -77.03230763506622]
158127     [38.91780661088507, -77.03207932431052]
158128     [38.88408977885549, -76.99517967895706]
158129      [38.90659875790153, -77.0514937164447]
Length: 5737818, dtype: object

In [15]:
df["temp_start_lat"] = lat_long_apply.str[0]
df["temp_start_lng"] = lat_long_apply.str[1]
df["temp_end_lat"] = lat_long_end_apply.str[0]
df["temp_end_lng"] = lat_long_end_apply.str[1]

In [16]:
df

,Duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,ride_id,member_casual,fileid,...,rideable_type,start_lat,start_lng,end_lat,end_lng,id,temp_start_lat,temp_start_lng,temp_end_lat,temp_end_lng
0,429.0,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,member,2019-10,...,NaN,NaN,NaN,NaN,NaN,1,38.912097,-77.038689,38.912097,-77.038689
1,1935.0,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,member,2019-10,...,NaN,NaN,NaN,NaN,NaN,2,38.887374,-77.001958,38.887374,-77.001958
2,563.0,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,member,2019-10,...,NaN,NaN,NaN,NaN,NaN,3,38.912097,-77.038689,38.912097,-77.038689
3,625.0,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,member,2019-10,...,NaN,NaN,NaN,NaN,NaN,4,38.938741,-77.087181,38.938741,-77.087181
4,312.0,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,member,2019-10,...,NaN,NaN,NaN,NaN,NaN,5,38.908644,-77.012366,38.908644,-77.012366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158125,886.0,2019-02-28 23:55:21,2019-03-01 00:10:08,31223,Convention Center / 7th & M St NW,31612,D St & Maryland Ave NE,W23129,member,2019-02,...,NaN,NaN,NaN,NaN,NaN,158126,38.905731,-77.022264,38.905731,-77.022264
158126,404.0,2019-02-28 23:57:03,2019-03-01 00:03:47,31203,14th & Rhode Island Ave NW,31223,Convention Center / 7th & M St NW,W00316,member,2019-02,...,NaN,NaN,NaN,NaN,NaN,158127,38.908596,-77.032308,38.908596,-77.032308
158127,972.0,2019-02-28 23:57:03,2019-03-01 00:13:16,31101,14th & V St NW,31600,5th & K St NW,W21796,member,2019-02,...,NaN,NaN,NaN,NaN,NaN,158128,38.917807,-77.032079,38.917807,-77.032079
158128,925.0,2019-02-28 23:57:32,2019-03-01 00:12:58,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31642,New Jersey Ave & L St NW,W23576,member,2019-02,...,NaN,NaN,NaN,NaN,NaN,158129,38.884090,-76.995180,38.884090,-76.995180


Done with adding new lat/long, 
now moving on to cleaning the table

In [20]:
for col in df.columns:
    print(col)

Duration
started_at
ended_at
start_station_id
start_station_name
end_station_id
end_station_name
ride_id
member_casual
fileid
year
month
rideable_type
start_lat
start_lng
end_lat
end_lng
id
temp_start_lat
temp_start_lng
temp_end_lat
temp_end_lng


In [22]:
# dropping unnecessary columns
df = df.drop(columns=['month','year','start_lat','start_lng','end_lat','end_lng'])

In [26]:
# renaming the new columns
df2 = df.rename(columns = {'temp_start_lat': 'new_start_lat', 'temp_start_lng': 'new_start_lng', 'temp_end_lat':'new_end_lat', 'temp_end_lng':'new_end_lng' }, inplace = False)
df2

In [ ]:
# save it to a new file if wanted
#df.to_csv(r'/Users/jennyjang/Desktop/CS Project/datasets/full_v2.csv')
